In [153]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
from scipy.special import softmax
import csv
import urllib.request



task='sentiment'
MODEL = f"distilbert-base-uncased-finetuned-sst-2-english"

tokenizer = AutoTokenizer.from_pretrained(MODEL, model_max_length=256)


Didn't find file distilbert-base-uncased-finetuned-sst-2-english\added_tokens.json. We won't load it.
loading file distilbert-base-uncased-finetuned-sst-2-english\vocab.txt
loading file distilbert-base-uncased-finetuned-sst-2-english\tokenizer.json
loading file None
loading file distilbert-base-uncased-finetuned-sst-2-english\special_tokens_map.json
loading file distilbert-base-uncased-finetuned-sst-2-english\tokenizer_config.json


In [154]:
from datasets import load_dataset

raw_datasets = load_dataset("dataset")

Using custom data configuration dataset-20a4292a64604c7b
Reusing dataset csv (C:\Users\samue\.cache\huggingface\datasets\csv\dataset-20a4292a64604c7b\0.0.0\6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e)
100%|██████████| 2/2 [00:00<00:00, 497.43it/s]


In [155]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 61004
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 26144
    })
})

In [156]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

100%|██████████| 27/27 [00:01<00:00, 17.39ba/s]


In [157]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(int(10000*0.7))) 
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=30).select(range(int(10000*0.3))) 
full_train_dataset = tokenized_datasets["train"]
full_eval_dataset = tokenized_datasets["test"]

In [158]:
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
model.save_pretrained(MODEL)
tokenizer.save_pretrained(MODEL)

loading configuration file distilbert-base-uncased-finetuned-sst-2-english\config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased-finetuned-sst-2-english",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "finetuning_task": "sst-2",
  "hidden_dim": 3072,
  "id2label": {
    "0": "NEGATIVE",
    "1": "POSITIVE"
  },
  "initializer_range": 0.02,
  "label2id": {
    "NEGATIVE": 0,
    "POSITIVE": 1
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torch_dtype": "float32",
  "transformers_version": "4.14.1",
  "vocab_size": 30522
}

loading weights file distilbert-base-uncased-finetuned-sst-2-english\pytorch_model.bin
All model checkpoint weights wer

('distilbert-base-uncased-finetuned-sst-2-english\\tokenizer_config.json',
 'distilbert-base-uncased-finetuned-sst-2-english\\special_tokens_map.json',
 'distilbert-base-uncased-finetuned-sst-2-english\\vocab.txt',
 'distilbert-base-uncased-finetuned-sst-2-english\\added_tokens.json',
 'distilbert-base-uncased-finetuned-sst-2-english\\tokenizer.json')

In [159]:
from transformers import TrainingArguments
import datetime
 
currentDT = datetime.datetime.now()

training_args = TrainingArguments(
    output_dir='./results/'+str(currentDT).replace(" ", "_")[:str(currentDT).index(".")].replace(":", "_"),
    evaluation_strategy="epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    weight_decay=0.001,
    warmup_steps=600
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [160]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [161]:
from transformers import Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=full_train_dataset,
    eval_dataset=full_eval_dataset,
    compute_metrics=compute_metrics,
)
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 26144
  Batch size = 32
100%|██████████| 817/817 [13:02<00:00,  1.04it/s]


{'eval_loss': 2.2577626705169678,
 'eval_accuracy': 0.5506808445532436,
 'eval_runtime': 782.8612,
 'eval_samples_per_second': 33.395,
 'eval_steps_per_second': 1.044}

In [162]:
model.save_pretrained(MODEL)

Configuration saved in distilbert-base-uncased-finetuned-sst-2-english\config.json
Model weights saved in distilbert-base-uncased-finetuned-sst-2-english\pytorch_model.bin
